In [5]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import os
from git import Repo

In [6]:
repo_path = '/Users/rafaeltiburcio/dev/python/rag/projeto2/test_repo'
repo = Repo.clone_from('https://github.com/langchain-ai/langchain', to_path=repo_path)

In [9]:
loader = GenericLoader.from_filesystem(
    repo_path + '/libs/core/langchain_core',
    glob='**/*',
    suffixes=[".py"],
    exclude=["**/non-utf-8-encoding.py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

documents = loader.load()
len(documents)

447

In [10]:
python_spliter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=2000,
    chunk_overlap=200
)

texts = python_spliter.split_documents(documents)
len(texts)

1359

In [11]:
os.environ['OPENAI_API_KEY'] = ""

In [12]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={ "k": 8 }
)

In [13]:
llm = ChatOpenAI(model='gpt-4o-mini', max_tokens=200)

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um revisor de código experiente, forneça informações detalhadas sobre a revisão do código e sugestões de melhorias baseado no contexto fornecido abaixo: \n\n {context}"
        ),
        (
            "user",
            "{input}"
        )
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({
    "input": "Você revisar e sugerir melhorias para o código de RunnableBinding"
})

In [15]:
print(response['answer'])

Claro! Vamos revisar o código da classe `RunnableLearnable`, que herda de `RunnableBinding`, e discutir algumas melhorias e sugestões.

### Revisão do Código

```python
from langchain_core.runnables.base import RunnableBinding

class RunnableLearnable(RunnableBinding):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.parameters = []

    def backward(self):
        for param in self.parameters:
            param.backward()

    def update(self, optimizer):
        for param in self.parameters:
            optimizer.update(param)
```

#### Pontos Positivos:
1. **Herança**: A classe `RunnableLearnable` está corretamente herança da classe `RunnableBinding`, o que permite a reutilização de código e a extensibilidade.
2. **Inicialização**: O uso do `*args` e `**kwargs` na inicialização permite que a classe seja flexível em relação aos parâmetros que aceita
